In [1]:
import json
import pandas as pd

In [1]:
# Ouvrir le fichier JSON
with open('export_epita.json') as fichier:
    # Charger les données JSON
    donnees = json.load(fichier)

# Utiliser les données
data = []
for doc in donnees:
    if doc['document_type_id'] == 8:
        for a in doc.values():
            if type(a) is list :
                data += a
                
data

[{'line': {'words': [{'text': 'Laboratoire',
     'width': 0.15389174222946167,
     'height': 0.012780756689608097,
     'left': 0.44787001609802246,
     'top': 0.04207148402929306},
    {'text': 'de',
     'width': 0.02989768795669079,
     'height': 0.01207193173468113,
     'left': 0.6138680577278137,
     'top': 0.042377110570669174},
    {'text': 'Biologie',
     'width': 0.11026273667812347,
     'height': 0.015476633794605732,
     'left': 0.6566705107688904,
     'top': 0.041830144822597504},
    {'text': 'Médicale',
     'width': 0.11637591570615768,
     'height': 0.012553628534078598,
     'left': 0.7790614366531372,
     'top': 0.041519396007061005}],
   'width': 0.44756796956062317,
   'height': 0.015980247408151627,
   'left': 0.4478699862957001,
   'top': 0.04138056933879852,
   'confidence': 99.16895294189453}},
 {'line': {'words': [{'text': 'Chemin',
     'width': 0.1001322939991951,
     'height': 0.0121104521676898,
     'left': 0.5884195566177368,
     'top': 0.06

In [2]:
# Open your JSON file and load it as a list of dictionaries
with open('export_epita.json') as f:
    data_list = json.load(f)

# Convert each dictionary to a JSON string and store in a new list
json_strings = [json.dumps(item) for item in data_list]

# Convert this list to a DataFrame
df = pd.DataFrame(json_strings, columns=['json_string'])

# Print the first few rows to check
print(df.head())

                                         json_string
0  {"0": [{"line": {"words": [{"text": "EU", "wid...
1  {"0": [{"line": {"words": [{"text": "SOCIALE",...
2  {"0": [{"line": {"words": [{"text": "Laboratoi...
3  {"0": [{"line": {"words": [{"text": "R\u00e9ca...
4  {"0": [{"line": {"words": [{"text": "CARTE", "...


In [3]:
def has_document_type_id(input_dict, document_type_id):
    if isinstance(input_dict, dict):
        for key, value in input_dict.items():
            if key == "document_type_id" and value == document_type_id:
                return True
            if isinstance(value, dict):
                if has_document_type_id(value, document_type_id):
                    return True
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        if has_document_type_id(item, document_type_id):
                            return True
    return False

df = df[df['json_string'].apply(lambda x: has_document_type_id(json.loads(x), 8))]

In [7]:
df.shape

(289, 1)

In [7]:
len(data)

47583

In [2]:
from transformers import pipeline

# Chargement du modèle pré-entraîné
nlp = pipeline("question-answering", model="distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to

In [3]:
def find_unit_value(entity_name, entities, index):
    value = None
    unit = None

    # Recherche de la valeur et de l'unité dans les lignes suivantes
    for next_index in range(index, len(data)):
        next_item = data[next_index]
        next_line = next_item["line"]
        next_words = [word["text"] for word in next_line["words"]]
        next_text = " ".join(next_words)

        # Recherche de la valeur dans la ligne
        if value is None:
            for i in range(len(next_words)):
                if next_words[i].replace(".", "", 1).isdigit():
                    value = next_words[i]
                    break

        # Recherche de l'unité dans la ligne
        if unit is None:
            for i in range(len(next_words)):
                # Faire un truc pour les unités par rapport au entity_name
                if "fl" in next_words[i].lower() or "g/dl" in next_words[i].lower() or "pg" in next_words[i].lower():
                    unit = next_words[i]
                    break

        # Si la valeur et l'unité ont été trouvées, sortie de la boucle
        if value and unit:
            break

    # Si la valeur et l'unité ont été trouvées, stockage des informations dans le dictionnaire des entités
    if value and unit:
        if entity_name in entities :
            entities[entity_name + str(len(entities.keys()))] = {"value": value, "unit": unit}
        else:
            entities[entity_name] = {"value": value, "unit": unit}

# Stockage des entités pertinentes et des informations associées
entities = {}

for index, item in enumerate(data):
    line = item["line"]
    words = [word["text"].lower() for word in line["words"]]
    text = " ".join(words)
    
    if "v.g.m" in words:
        find_unit_value("v.g.m", entities, index)
            
    elif "hémoglobine" in words:
        find_unit_value("hémoglobine", entities, index)
    
    elif "t.c.m.h" in words:
        find_unit_value("t.c.m.h", entities, index)
        
    elif "c.c.m.h" in words:
        find_unit_value("c.c.m.h", entities, index)

# Affichage des informations extraites
for entity, info in entities.items():
    print(f"{entity}:")
    print(f"Valeur: {info['value']}")
    print(f"Unité: {info['unit']}")
    print()

hémoglobine:
Valeur: 39
Unité: pg

hémoglobine1:
Valeur: 94
Unité: g/dL

hémoglobine2:
Valeur: 85
Unité: pg

v.g.m:
Valeur: 85
Unité: pg

c.c.m.h:
Valeur: 1
Unité: pg

t.c.m.h:
Valeur: 1
Unité: pg

hémoglobine6:
Valeur: 34
Unité: pg

hémoglobine7:
Valeur: 34
Unité: pg

hémoglobine8:
Valeur: 16.0
Unité: g/dL

hémoglobine9:
Valeur: 89
Unité: pg

hémoglobine10:
Valeur: 95
Unité: g/dL

hémoglobine11:
Valeur: 11.5
Unité: g/dl

hémoglobine12:
Valeur: 92
Unité: pg

hémoglobine13:
Valeur: 11.5
Unité: g/dL

hémoglobine14:
Valeur: 89
Unité: pg/l

v.g.m15:
Valeur: 89
Unité: pg/l

c.c.m.h16:
Valeur: 35.6
Unité: pg/l

t.c.m.h17:
Valeur: 35.6
Unité: pg/l

hémoglobine18:
Valeur: 87
Unité: g/dL

hémoglobine19:
Valeur: 139
Unité: g/dL

hémoglobine20:
Valeur: 13.0
Unité: g/dL

c.c.m.h21:
Valeur: 31.0
Unité: g/dL

hémoglobine22:
Valeur: 91
Unité: g/dL

v.g.m23:
Valeur: 91
Unité: fL

t.c.m.h24:
Valeur: 8500
Unité: pg

c.c.m.h25:
Valeur: 8500
Unité: g/dL

hémoglobine26:
Valeur: 35
Unité: pg

hémoglobine27: